<a href="https://colab.research.google.com/github/bryanbayup/phising-detection/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!unzip allinone.zip

Archive:  allinone.zip
   creating: app/
   creating: app/models/
  inflating: app/models/model_intent.keras  
  inflating: app/models/model_ner.keras  
   creating: app/data/
  inflating: app/data/vectorizer.pickle  
  inflating: app/data/stopword_list_tala.txt  
  inflating: app/data/dataaa.json    
   creating: app/encoders/
  inflating: app/encoders/tokenizer.pickle  
  inflating: app/encoders/ner_label_encoder.pickle  
  inflating: app/encoders/label_encoder.pickle  


In [4]:
!pip install datasets seqeval imblearn ipywidgets
!jupyter nbextension enable --py widgetsnbextension

  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)
  Using cached seqeval-1.2.2.tar.gz (43 kB)
  Preparing metadata (setup.py) ... done
  Using cached imblearn-0.0-py2.py3-none-any.whl.metadata (355 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.6 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=adc99b5ac7d4776f969ca4026067a83ffc98755024c1c3145544c49c85c48957
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
  Attempting 

In [5]:
#Download dan ekstrak fastext bahasa indonesia
!wget -O id.tar.gz "https://www.dropbox.com/scl/fi/sju4o3keikox69euw51vy/id.tar.gz?rlkey=5jr3ijtbdwfahq7xcgig28qvy&e=1&st=gntzkzeo&dl=1"

--2024-12-06 06:43:37--  https://www.dropbox.com/scl/fi/sju4o3keikox69euw51vy/id.tar.gz?rlkey=5jr3ijtbdwfahq7xcgig28qvy&e=1&st=gntzkzeo&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc77a697554a1e710b95971e8d08.dl.dropboxusercontent.com/cd/0/inline/CftCuqQp_46M-uv31pTXYtqAu84yhdgwSZ34ltBaWsmmYncoDfr9_B3-gHRkSJo-L4YD5KUr4d3e5sU9u0XWRq-deh0ZszXg3ctfMSW-QjFqstDOO6Q9BzIb0LjbTw5Jecs/file?dl=1# [following]
--2024-12-06 06:43:38--  https://uc77a697554a1e710b95971e8d08.dl.dropboxusercontent.com/cd/0/inline/CftCuqQp_46M-uv31pTXYtqAu84yhdgwSZ34ltBaWsmmYncoDfr9_B3-gHRkSJo-L4YD5KUr4d3e5sU9u0XWRq-deh0ZszXg3ctfMSW-QjFqstDOO6Q9BzIb0LjbTw5Jecs/file?dl=1
Resolving uc77a697554a1e710b95971e8d08.dl.dropboxusercontent.com (uc77a697554a1e710b95971e8d08.dl.dropboxusercontent.com)... 162.125.81.15, 2620:100:6031:15

In [6]:
!tar -xzf id.tar.gz

In [7]:
import json
import os
import re
import pickle
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import nltk
from IPython.display import display
import ipywidgets as widgets

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Bidirectional, LSTM, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from datasets import Dataset

# Unduh stopwords jika diperlukan
nltk.download('stopwords')
# Set Direktori Kerja
os.chdir('/content/app')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
!ls

data  encoders	models


In [10]:
os.makedirs('data', exist_ok=True)
data_path = 'data/data2.json'  # Ganti dengan path dataset baru Anda

with open(data_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

utterances = []
intents = []
entities_all = []

for conv in data:
    for turn in conv['turns']:
        if turn['speaker'] == 'user':
            utt = turn['utterance']
            intent = turn['intent']
            ents = turn.get('entities', [])
            utterances.append(utt)
            intents.append(intent)
            entities_all.append(ents)

df = pd.DataFrame({'utterance': utterances, 'intent': intents, 'entities': entities_all})
print("Contoh data intent & NER:")
print(df.head())

Contoh data intent & NER:
                                           utterance  \
0    Saya melihat seekor kucing sakit di depan toko.   
1           Kucing terlihat demam dan bersin-bersin.   
2                  Ada anjing terluka di jalan raya.   
3              Ya, anjing muntah dan terlihat lemas.   
4  Saya menemukan seekor kucing dengan luka di ka...   

                       intent  \
0  Melaporkan Hewan Terlantar   
1         Mendiagnosis Gejala   
2  Melaporkan Hewan Terlantar   
3         Mendiagnosis Gejala   
4  Melaporkan Hewan Terlantar   

                                            entities  
0  [{'entity': 'animal', 'value': 'kucing', 'star...  
1  [{'entity': 'animal', 'value': 'kucing', 'star...  
2  [{'entity': 'animal', 'value': 'anjing', 'star...  
3  [{'entity': 'animal', 'value': 'anjing', 'star...  
4  [{'entity': 'animal', 'value': 'kucing', 'star...  


In [11]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

df['utterance_clean'] = df['utterance'].apply(clean_text)

In [12]:
label_encoder = LabelEncoder()
df['intent_label'] = label_encoder.fit_transform(df['intent'])
num_classes = len(label_encoder.classes_)

os.makedirs('encoders', exist_ok=True)
with open('encoders/label_encoder_new.pickle', 'wb') as f:
    pickle.dump(label_encoder, f)

texts = df['utterance_clean'].tolist()
labels = df['intent_label'].tolist()

train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

tokenizer_intent = tf.keras.preprocessing.text.Tokenizer(oov_token='')
tokenizer_intent.fit_on_texts(train_texts)
word_index = tokenizer_intent.word_index
vocab_size = len(word_index)+1

train_seq = tokenizer_intent.texts_to_sequences(train_texts)
val_seq = tokenizer_intent.texts_to_sequences(val_texts)

max_seq_length = max(max(len(s) for s in train_seq), max(len(s) for s in val_seq))
train_padded = pad_sequences(train_seq, maxlen=max_seq_length, padding='post')
val_padded = pad_sequences(val_seq, maxlen=max_seq_length, padding='post')

train_labels_cat = to_categorical(train_labels, num_classes=num_classes)
val_labels_cat = to_categorical(val_labels, num_classes=num_classes)

with open('encoders/tokenizer_intent_new.pickle', 'wb') as f:
    pickle.dump(tokenizer_intent, f)

In [16]:
#Load Fastext Bahasa Indonesia
fasttext_path = '/content/id.vec'
fasttext_model = {}
with open(fasttext_path, 'r', encoding='utf-8') as f:
    first_line = True
    for line in f:
        if first_line:
            first_line = False
            continue
        values = line.strip().split()
        word = values[0]
        vector = np.array(values[1:], dtype='float32')
        fasttext_model[word] = vector
embedding_dim = len(vector)  # misalnya 300

embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, idx in word_index.items():
    if word in fasttext_model:
        embedding_matrix[idx] = fasttext_model[word]
    else:
        embedding_matrix[idx] = np.random.normal(scale=0.6, size=(embedding_dim,))

In [20]:
def build_intent_model(embedding_matrix, max_seq_length, num_classes, l2_reg=0.001):
    inputs = Input(shape=(max_seq_length,))
    embedding = Embedding(
        input_dim=embedding_matrix.shape[0],
        output_dim=embedding_matrix.shape[1],
        weights=[embedding_matrix],
        input_length=max_seq_length,
        trainable=False  # Frozen embedding agar fokus ke training layer atas
    )(inputs)
    conv = Conv1D(filters=128, kernel_size=3, activation='relu')(embedding)
    pool = GlobalMaxPooling1D()(conv)
    dense = Dense(64, activation='relu', kernel_regularizer=l2(l2_reg))(pool)
    drop = Dropout(0.5)(dense)
    outputs = Dense(num_classes, activation='softmax')(drop)
    model = Model(inputs, outputs)
    return model

model_intent = build_intent_model(embedding_matrix, max_seq_length, num_classes)
model_intent.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Anda dapat meningkatkan epoch, batch size, atau menggunakan callback seperti EarlyStopping
model_intent.fit(
    train_padded, train_labels_cat,
    validation_data=(val_padded, val_labels_cat),
    epochs=5,
    batch_size=16
)

os.makedirs('models', exist_ok=True)
model_intent.save('models/model_intent_new.keras')

Epoch 1/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 259ms/step - accuracy: 0.2699 - loss: 1.6178 - val_accuracy: 0.4667 - val_loss: 1.3415
Epoch 2/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5896 - loss: 1.1684 - val_accuracy: 0.4667 - val_loss: 1.2504
Epoch 3/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6203 - loss: 1.1202 - val_accuracy: 0.5667 - val_loss: 1.1117
Epoch 4/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7561 - loss: 0.8241 - val_accuracy: 0.6000 - val_loss: 1.0847
Epoch 5/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7296 - loss: 0.7719 - val_accuracy: 0.6000 - val_loss: 0.9839


In [21]:
all_labels = set(['O'])
for ents in df['entities']:
    for ent in ents:
        ent_type = ent['entity']
        all_labels.add('B-' + ent_type)
        all_labels.add('I-' + ent_type)

ner_label_encoder = {label: idx for idx, label in enumerate(sorted(all_labels))}
ner_label_decoder = {idx: label for label, idx in ner_label_encoder.items()}

with open('encoders/ner_label_encoder_new.pickle', 'wb') as f:
    pickle.dump(ner_label_encoder, f)

def create_ner_data(texts, entities_list, tokenizer, max_len):
    X = []
    Y = []
    for text, ents in zip(texts, entities_list):
        words = text.split()
        seq = tokenizer.texts_to_sequences([text])[0]
        label_seq = ['O'] * len(seq)
        # Sederhana, untuk NER yang lebih robust, pertimbangkan alignment token subword.
        for ent in ents:
            ent_text = clean_text(ent['value'])
            ent_tokens = ent_text.split()
            for i in range(len(words)-len(ent_tokens)+1):
                if words[i:i+len(ent_tokens)] == ent_tokens:
                    label_seq[i] = 'B-' + ent['entity']
                    for j in range(1, len(ent_tokens)):
                        label_seq[i+j] = 'I-' + ent['entity']
                    break
        if len(seq) < max_len:
            seq += [0]*(max_len-len(seq))
            label_seq += ['O']*(max_len-len(label_seq))
        else:
            seq = seq[:max_len]
            label_seq = label_seq[:max_len]

        X.append(seq)
        Y.append([ner_label_encoder[l] for l in label_seq])

    X = np.array(X)
    Y = np.array(Y)
    Y = to_categorical(Y, num_classes=len(ner_label_encoder))
    return X, Y

X_ner, Y_ner = create_ner_data(df['utterance_clean'].tolist(), df['entities'].tolist(), tokenizer_intent, max_seq_length)
X_ner_train, X_ner_val, Y_ner_train, Y_ner_val = train_test_split(X_ner, Y_ner, test_size=0.2, random_state=42)

def build_ner_model(embedding_matrix, max_seq_length, num_entities):
    inputs = Input(shape=(max_seq_length,))
    embedding = Embedding(
        input_dim=embedding_matrix.shape[0],
        output_dim=embedding_matrix.shape[1],
        weights=[embedding_matrix],
        input_length=max_seq_length,
        trainable=False
    )(inputs)
    lstm = Bidirectional(LSTM(128, return_sequences=True))(embedding)
    drop = Dropout(0.5)(lstm)
    outputs = TimeDistributed(Dense(num_entities, activation='softmax'))(drop)
    model = Model(inputs, outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model_ner = build_ner_model(embedding_matrix, max_seq_length, len(ner_label_encoder))
model_ner.fit(
    X_ner_train, Y_ner_train,
    validation_data=(X_ner_val, Y_ner_val),
    epochs=5,
    batch_size=16
)

model_ner.save('models/model_ner_new.keras')

Epoch 1/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 112ms/step - accuracy: 0.6134 - loss: 2.0362 - val_accuracy: 0.8104 - val_loss: 1.1225
Epoch 2/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.8611 - loss: 0.8993 - val_accuracy: 0.8104 - val_loss: 0.9607
Epoch 3/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8770 - loss: 0.6014 - val_accuracy: 0.8375 - val_loss: 0.7293
Epoch 4/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8730 - loss: 0.5804 - val_accuracy: 0.8417 - val_loss: 0.6346
Epoch 5/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8884 - loss: 0.4958 - val_accuracy: 0.8417 - val_loss: 0.5769


In [22]:
lm_texts = []
for conv in data:
    for turn in conv['turns']:
        prefix = "User:" if turn['speaker'] == 'user' else "Bot:"
        lm_texts.append(f"{prefix} {turn['utterance']}")
    lm_texts.append("")

lm_corpus = "\n".join(lm_texts)
with open('data/lm_corpus.txt', 'w', encoding='utf-8') as f:
    f.write(lm_corpus)

gpt_model_name = "cahya/gpt2-small-indonesian-522M"
gpt_tokenizer = AutoTokenizer.from_pretrained(gpt_model_name)
gpt_model = AutoModelForCausalLM.from_pretrained(gpt_model_name)

# Tambahkan PAD token jika belum ada
if gpt_tokenizer.pad_token is None:
    gpt_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    gpt_model.resize_token_embeddings(len(gpt_tokenizer))

lines = lm_corpus.splitlines()
dataset = Dataset.from_dict({'text': lines})

def tokenize_function(examples):
    return gpt_tokenizer(examples["text"], truncation=True, max_length=128, padding='max_length')

tokenized_dataset = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=gpt_tokenizer,
    mlm=False,
    return_tensors="pt"
)

# Gunakan GPU jika tersedia untuk mempercepat
# Sesuaikan epoch dan batch size untuk hasil lebih baik
training_args = TrainingArguments(
    output_dir="./gpt_finetuned",
    overwrite_output_dir=True,
    per_device_train_batch_size=2,
    num_train_epochs=1,  # Tingkatkan epoch sesuai kebutuhan
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    logging_dir='./logs',
    do_train=True,
    do_eval=True
)

tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = tokenized_dataset['train']
eval_dataset = tokenized_dataset['test']

trainer = Trainer(
    model=gpt_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator
)

trainer.train()
trainer.save_model("./gpt_finetuned")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/894k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/452k [00:00<?, ?B/s]

...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
100,3.256100


In [23]:
with open('encoders/label_encoder_new.pickle', 'rb') as f:
    label_encoder_intent = pickle.load(f)

with open('encoders/ner_label_encoder_new.pickle', 'rb') as f:
    ner_label_encoder = pickle.load(f)
ner_label_decoder = {v: k for k, v in ner_label_encoder.items()}

def predict_intent_fn(utterance):
    seq = tokenizer_intent.texts_to_sequences([utterance])
    padded_seq = pad_sequences(seq, maxlen=max_seq_length, padding='post')
    predictions = model_intent.predict(padded_seq)
    intent = label_encoder_intent.inverse_transform([predictions.argmax(axis=1)[0]])
    return intent[0]

def predict_ner_fn(utterance):
    words = utterance.split()
    seq = tokenizer_intent.texts_to_sequences([utterance])[0]
    padded_seq = pad_sequences([seq], maxlen=max_seq_length, padding='post')
    predictions = model_ner.predict(padded_seq).argmax(axis=-1)[0]
    labels = [ner_label_decoder[p] for p in predictions[:len(words)]]
    return list(zip(words, labels))

def generate_response(context, max_length=50):
    input_ids = gpt_tokenizer(context, return_tensors='pt').input_ids
    input_ids = input_ids.to(gpt_model.device)
    gen_tokens = gpt_model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        temperature=0.9,
        do_sample=True # do_sample True untuk menggunakan temperature
    )
    gen_text = gpt_tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)[0]
    return gen_text

In [24]:
input_box = widgets.Text(
    placeholder='Masukkan teks...',
    description='User Input:',
)

output_box = widgets.Output()

def on_button_click(b):
    user_input = input_box.value
    with output_box:
        output_box.clear_output()
        print(f"User Input: {user_input}")
        pred_intent = predict_intent_fn(user_input)
        print(f"Predicted Intent: {pred_intent}")
        ner_res = predict_ner_fn(user_input)
        print("NER Result:")
        for w,l in ner_res:
            print(f" {w} -> {l}")
        # Buat konteks sederhana
        context = f"User: {user_input}\nBot:"
        response = generate_response(context)
        print(f"GPT Response: {response}")

button = widgets.Button(description="Proses")
button.on_click(on_button_click)

display(input_box, button, output_box)

Text(value='', description='User Input:', placeholder='Masukkan teks...')

Button(description='Proses', style=ButtonStyle())

Output()